In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
from collections import Counter
import numpy as np
import sys
import matplotlib.pyplot as plt

# LOAD DATA

In [2]:
df_usr_view_pay = pd.read_csv('dataset/df_view_pay_with_month_week_aqi_weather.csv', index_col=[0, 1])
df_usr_view_pay

view_cnt  pay_cnt  weekday  month  aqi  weather  wind  \
shop_id time_stamp                                                          
1       2015-10-10         0      188        5     10   64      0.0   0.0   
        2015-10-11         0      272        6     10   68      6.0   0.0   
        2015-10-12         0      257        0     10   85      1.0   0.0   
        2015-10-13         0      239        1     10   84      5.0   0.0   
        2015-10-14         0      277        2     10  143      0.0   0.0   
        2015-10-15         0      227        3     10  164      6.0   0.0   
        2015-10-16         0      242        4     10  135      1.0   0.0   
        2015-10-17         0      293        5     10   78      5.0   0.0   
        2015-10-18         0      291        6     10   66      5.0   0.0   
        2015-10-19         0      256        0     10   67      0.0   0.0   
        2015-10-20         0      262        1     10   76      7.0   0.0   
        2015-10-21         0      271        2     10   69      0.0   0.0   
        2015-10-22         0      260        3     10   85      0.0   0.0   
        2015-10-23         0      240        4     10  102      7.0   0.0   
        2015-10-24         0      291        5     10   77      3.0   0.0   
        2015-10-25         0      286        6     10   71      7.0   0.0   
        2015-10-26         0      264        0     10   83     27.0   0.0   
        2015-10-27         0      295        1     10   66      8.0   0.0   
        2015-10-28         0      310        2     10   86     16.0   0.0   
        2015-10-29         0      212        3     10   76      4.0   0.0   
        2015-10-30         0      331        4     10   51      3.0   0.0   
        2015-10-31         0      347        5     10   40     16.0   0.0   
        2015-11-01         0      381        6     11   66      8.0   0.0   
        2015-11-02         0      332        0     11   80      5.0   0.0   
        2015-11-03         0      404        1     11  111      5.0   0.0   
        2015-11-04         0      392        2     11  112      3.0   0.0   
        2015-11-05         0      496        3     11   89      8.0   0.0   
        2015-11-06         0      430        4     11   63     16.0   0.0   
        2015-11-07         0      351        5     11   46      4.0   0.0   
        2015-11-08         0      288        6     11   42      4.0   0.0   
...                      ...      ...      ...    ...  ...      ...   ...   
2000    2016-10-02         4       39        6     10   65      0.0   2.0   
        2016-10-03         3       50        0     10   37      4.0   5.0   
        2016-10-04         8       29        1     10   46      0.0   1.0   
        2016-10-05         3       35        2     10   56      0.0   1.0   
        2016-10-06         4       76        3     10   54      4.0   5.0   
        2016-10-07        11       68        4     10   43      4.0   2.0   
        2016-10-08        16      158        5     10   36      2.0   5.0   
        2016-10-09        24      176        6     10   42      0.0   5.0   
        2016-10-10        35      161        0     10   42      3.0   2.0   
        2016-10-11        24      163        1     10   46      3.0   2.0   
        2016-10-12        33      159        2     10   52      0.0   2.0   
        2016-10-13        35      177        3     10   67      0.0   2.0   
        2016-10-14        14      148        4     10   58      2.0   2.0   
        2016-10-15         9       78        5     10   55      2.0   1.0   
        2016-10-16         4       60        6     10   60      2.0   2.0   
        2016-10-17        16      163        0     10   80      2.0   2.0   
        2016-10-18        18      157        1     10   61      2.0   1.0   
        2016-10-19        14      164        2     10   50      2.0   1.0   
        2016-10-20        35      165        3     10   40      4.0   2.0   
        2016-10-

In [3]:
df_usr_view_pay.columns

Index(['view_cnt', 'pay_cnt', 'weekday', 'month', 'aqi', 'weather', 'wind',
       'tmp'],
      dtype='object')

In [4]:
quant_features = ['view_cnt', 'pay_cnt', 'weekday', 'month', 'aqi', 'weather', 'wind', 'tmp']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = df_usr_view_pay[each].mean(), df_usr_view_pay[each].std()
    scaled_features[each] = [mean, std]
    df_usr_view_pay.loc[:, each] = (df_usr_view_pay[each] - mean)/std
scaled_features

{'aqi': [69.10689907646486, 41.82863241974223],
 'month': [6.743497867987794, 3.1398223895350226],
 'pay_cnt': [108.25559585194213, 130.7345540633734],
 'tmp': [20.06643747988851, 9.000948791616628],
 'view_cnt': [15.710891994111511, 42.47333460898241],
 'weather': [12.96204816186815, 29.02202674421276],
 'weekday': [3.0034805507322524, 2.003086129482419],
 'wind': [1.3992824375980313, 2.446529545336942]}

In [5]:
# df_usr_view_pay = df_usr_view_pay.drop(['time_stamp', 'view_cnt'], axis=1)
df_usr_view_pay

view_cnt   pay_cnt   weekday     month       aqi  \
shop_id time_stamp                                                     
1       2015-10-10 -0.369900  0.609972  0.996722  1.037161 -0.122091   
        2015-10-11 -0.369900  1.252495  1.495951  1.037161 -0.026463   
        2015-10-12 -0.369900  1.137759 -1.499427  1.037161  0.379957   
        2015-10-13 -0.369900  1.000075 -1.000197  1.037161  0.356050   
        2015-10-14 -0.369900  1.290741 -0.500967  1.037161  1.766567   
        2015-10-15 -0.369900  0.908286 -0.001738  1.037161  2.268616   
        2015-10-16 -0.369900  1.023023  0.497492  1.037161  1.575311   
        2015-10-17 -0.369900  1.413126  0.996722  1.037161  0.212608   
        2015-10-18 -0.369900  1.397828  1.495951  1.037161 -0.074277   
        2015-10-19 -0.369900  1.130110 -1.499427  1.037161 -0.050370   
        2015-10-20 -0.369900  1.176004 -1.000197  1.037161  0.164794   
        2015-10-21 -0.369900  1.244846 -0.500967  1.037161 -0.002556   
        2015-10-22 -0.369900  1.160706 -0.001738  1.037161  0.379957   
        2015-10-23 -0.369900  1.007724  0.497492  1.037161  0.786378   
        2015-10-24 -0.369900  1.397828  0.996722  1.037161  0.188701   
        2015-10-25 -0.369900  1.359582  1.495951  1.037161  0.045258   
        2015-10-26 -0.369900  1.191303 -1.499427  1.037161  0.332143   
        2015-10-27 -0.369900  1.428424 -1.000197  1.037161 -0.074277   
        2015-10-28 -0.369900  1.543161 -0.500967  1.037161  0.403865   
        2015-10-29 -0.369900  0.793550 -0.001738  1.037161  0.164794   
        2015-10-30 -0.369900  1.703791  0.497492  1.037161 -0.432883   
        2015-10-31 -0.369900  1.826177  0.996722  1.037161 -0.695861   
        2015-11-01 -0.369900  2.086246  1.495951  1.355651 -0.074277   
        2015-11-02 -0.369900  1.711440 -1.499427  1.355651  0.260422   
        2015-11-03 -0.369900  2.262175 -1.000197  1.355651  1.001541   
        2015-11-04 -0.369900  2.170386 -0.500967  1.355651  1.025448   
        2015-11-05 -0.369900  2.965891 -0.001738  1.355651  0.475586   
        2015-11-06 -0.369900  2.461051  0.497492  1.355651 -0.145998   
        2015-11-07 -0.369900  1.856773  0.996722  1.355651 -0.552418   
        2015-11-08 -0.369900  1.374881  1.495951  1.355651 -0.648047   
...                      ...       ...       ...       ...       ...   
2000    2016-10-02 -0.275723 -0.529742  1.495951  1.037161 -0.098184   
        2016-10-03 -0.299268 -0.445602 -1.499427  1.037161 -0.767582   
        2016-10-04 -0.181547 -0.606233 -1.000197  1.037161 -0.552418   
        2016-10-05 -0.299268 -0.560338 -0.500967  1.037161 -0.313348   
        2016-10-06 -0.275723 -0.246726 -0.001738  1.037161 -0.361162   
        2016-10-07 -0.110914 -0.307919  0.497492  1.037161 -0.624139   
        2016-10-08  0.006807  0.380499  0.996722  1.037161 -0.791489   
        2016-10-09  0.195160  0.518183  1.495951  1.037161 -0.648047   
        2016-10-10  0.454146  0.403447 -1.499427  1.037161 -0.648047   
        2016-10-11  0.195160  0.418745 -1.000197  1.037161 -0.552418   
        2016-10-12  0.407058  0.388148 -0.500967  1.037161 -0.408976   
        2016-10-13  0.454146  0.525832 -0.001738  1.037161 -0.050370   
        2016-10-14 -0.040282  0.304008  0.497492  1.037161 -0.265533   
        2016-10-15 -0.158002 -0.231428  0.996722  1.037161 -0.337255   
        2016-10-16 -0.275723 -0.369111  1.495951  1.037161 -0.217719   
        2016-10-17  0.006807  0.418745 -1.499427  1.037161  0.260422   
        2016-10-18  0.053895  0.372850 -1.000197  1.037161 -0.193812   
        2016-10-19 -0.040282  0.426394 -0.500967  1.037161 -0.456790   
        2016-10-20  0.454146  0.434043 -0.001738  1.037161 -0.695861   
        2016-10-21 -0.063826  0.227518  0.497492  1.037161 -0.863210   
        2016-10-22 -0.228635 -0.323217  0.996722  1.037161 -1.150095   
        2016-10-23 -0.275723 -0.376760  1.495951  1.037161 -0.743675   
        2016-10-24  0.077439  0.411096 -1.499427  1.037161 -

In [6]:
df_shop_info = pd.read_csv('dataset/normed_shop_info_features.csv', index_col=[0])
df_shop_info

,city_idx,location_id,per_pay,score,comment_cnt,shop_level_0,shop_level_1,shop_level_2,cate_1_idx_0,cate_1_idx_1,...,cate_3_idx_34,cate_3_idx_35,cate_3_idx_36,cate_3_idx_37,cate_3_idx_38,cate_3_idx_39,cate_3_idx_40,cate_3_idx_41,cate_3_idx_42,cate_3_idx_43
1,1.192024,0.899196,-0.430197,1.397435e+00,2.742265e+00,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1.402179,-1.545979,1.478704,-1.501043e-14,4.805603e-15,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.403944,0.568606,-0.950806,3.411693e-01,-3.495221e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0.561560,-0.604840,1.305167,-1.501043e-14,4.805603e-15,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,-0.594291,-0.953299,-1.471415,-7.150959e-01,-3.495221e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0.666637,1.655681,0.437485,3.411693e-01,-6.587009e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,-0.016366,-1.304737,0.437485,3.411693e-01,1.814729e+00,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,-0.594291,-1.039670,0.958095,3.411693e-01,-6.587009e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,-0.594291,1.599093,0.784558,1.397435e+00,-9.678796e-01,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
10,1.664872,-1.444717,-0.083124,-1.501043e-14,4.805603e-15,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


# SPLIT DATA TO TWO SETS

In [7]:
validation_ratio = 0.1
num_samples = 2000

In [8]:
validation_idx = np.random.randint(1, num_samples+1, size=int(validation_ratio*num_samples))
print(validation_idx.shape)
validation_idx

(200,)


array([1761, 1900, 1203,  184, 1706,  487, 1366, 1295,  546, 1317,  960,
        367, 1891,  138, 1285,  915,  490,  722, 1654,  927, 1966, 1883,
       1888,  348, 1335, 1013,  386,  995, 1446,  852, 1474, 1248, 1138,
       1036,  591, 1741,  108, 1007, 1210, 1148, 1121,  163, 1114,  696,
        162,  526, 1923,  231,  491,  719, 1143, 1658, 1377,  127,  589,
       1496, 1498,  576,  396, 1823, 1173, 1103,  399, 1233, 1844,  623,
         59,  705,  508,  417,  943,  867,  527, 1547,  974, 1339,  491,
       1209,  910, 1408, 1252, 1406,  660,   92, 1592,  731, 1070,  158,
       1527,  549,  411,  138,  659, 1039, 1595, 1378,  838,  454,  696,
        586,  197, 1847,  684, 1151,  584, 1603, 1252, 1125, 1662,  363,
        569, 1396,  670, 1006,  801,  674,   44, 1342, 1477,   82, 1637,
        914, 1966, 1149,  888, 1801, 1012,  942,  463, 1560,  577,  445,
       1151, 1376,  223,   89, 1642,  740, 1391,  324, 1618,  227,  606,
       1691,  897,  750,  163, 1656,  817, 1910, 13

In [9]:
len(set(validation_idx))

192

In [10]:
df_usr_train = df_usr_view_pay.drop(list(validation_idx), axis=0)
df_usr_train.reset_index(inplace=True)
df_usr_train.set_index(keys=['shop_id', 'time_stamp'], inplace=True)
df_usr_train

view_cnt   pay_cnt   weekday     month       aqi  \
shop_id time_stamp                                                     
1       2015-10-10 -0.369900  0.609972  0.996722  1.037161 -0.122091   
        2015-10-11 -0.369900  1.252495  1.495951  1.037161 -0.026463   
        2015-10-12 -0.369900  1.137759 -1.499427  1.037161  0.379957   
        2015-10-13 -0.369900  1.000075 -1.000197  1.037161  0.356050   
        2015-10-14 -0.369900  1.290741 -0.500967  1.037161  1.766567   
        2015-10-15 -0.369900  0.908286 -0.001738  1.037161  2.268616   
        2015-10-16 -0.369900  1.023023  0.497492  1.037161  1.575311   
        2015-10-17 -0.369900  1.413126  0.996722  1.037161  0.212608   
        2015-10-18 -0.369900  1.397828  1.495951  1.037161 -0.074277   
        2015-10-19 -0.369900  1.130110 -1.499427  1.037161 -0.050370   
        2015-10-20 -0.369900  1.176004 -1.000197  1.037161  0.164794   
        2015-10-21 -0.369900  1.244846 -0.500967  1.037161 -0.002556   
        2015-10-22 -0.369900  1.160706 -0.001738  1.037161  0.379957   
        2015-10-23 -0.369900  1.007724  0.497492  1.037161  0.786378   
        2015-10-24 -0.369900  1.397828  0.996722  1.037161  0.188701   
        2015-10-25 -0.369900  1.359582  1.495951  1.037161  0.045258   
        2015-10-26 -0.369900  1.191303 -1.499427  1.037161  0.332143   
        2015-10-27 -0.369900  1.428424 -1.000197  1.037161 -0.074277   
        2015-10-28 -0.369900  1.543161 -0.500967  1.037161  0.403865   
        2015-10-29 -0.369900  0.793550 -0.001738  1.037161  0.164794   
        2015-10-30 -0.369900  1.703791  0.497492  1.037161 -0.432883   
        2015-10-31 -0.369900  1.826177  0.996722  1.037161 -0.695861   
        2015-11-01 -0.369900  2.086246  1.495951  1.355651 -0.074277   
        2015-11-02 -0.369900  1.711440 -1.499427  1.355651  0.260422   
        2015-11-03 -0.369900  2.262175 -1.000197  1.355651  1.001541   
        2015-11-04 -0.369900  2.170386 -0.500967  1.355651  1.025448   
        2015-11-05 -0.369900  2.965891 -0.001738  1.355651  0.475586   
        2015-11-06 -0.369900  2.461051  0.497492  1.355651 -0.145998   
        2015-11-07 -0.369900  1.856773  0.996722  1.355651 -0.552418   
        2015-11-08 -0.369900  1.374881  1.495951  1.355651 -0.648047   
...                      ...       ...       ...       ...       ...   
2000    2016-10-02 -0.275723 -0.529742  1.495951  1.037161 -0.098184   
        2016-10-03 -0.299268 -0.445602 -1.499427  1.037161 -0.767582   
        2016-10-04 -0.181547 -0.606233 -1.000197  1.037161 -0.552418   
        2016-10-05 -0.299268 -0.560338 -0.500967  1.037161 -0.313348   
        2016-10-06 -0.275723 -0.246726 -0.001738  1.037161 -0.361162   
        2016-10-07 -0.110914 -0.307919  0.497492  1.037161 -0.624139   
        2016-10-08  0.006807  0.380499  0.996722  1.037161 -0.791489   
        2016-10-09  0.195160  0.518183  1.495951  1.037161 -0.648047   
        2016-10-10  0.454146  0.403447 -1.499427  1.037161 -0.648047   
        2016-10-11  0.195160  0.418745 -1.000197  1.037161 -0.552418   
        2016-10-12  0.407058  0.388148 -0.500967  1.037161 -0.408976   
        2016-10-13  0.454146  0.525832 -0.001738  1.037161 -0.050370   
        2016-10-14 -0.040282  0.304008  0.497492  1.037161 -0.265533   
        2016-10-15 -0.158002 -0.231428  0.996722  1.037161 -0.337255   
        2016-10-16 -0.275723 -0.369111  1.495951  1.037161 -0.217719   
        2016-10-17  0.006807  0.418745 -1.499427  1.037161  0.260422   
        2016-10-18  0.053895  0.372850 -1.000197  1.037161 -0.193812   
        2016-10-19 -0.040282  0.426394 -0.500967  1.037161 -0.456790   
        2016-10-20  0.454146  0.434043 -0.001738  1.037161 -0.695861   
        2016-10-21 -0.063826  0.227518  0.497492  1.037161 -0.863210   
        2016-10-22 -0.228635 -0.323217  0.996722  1.037161 -1.150095   
        2016-10-23 -0.275723 -0.376760  1.495951  1.037161 -0.743675   
        2016-10-24  0.077439  0.411096 -1.499427  1.037161 -

In [11]:
df_usr_train.index.levels[0]

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1989, 1990, 1992, 1993, 1994, 1995, 1996, 1998, 1999, 2000],
           dtype='int64', name='shop_id', length=1808)

In [12]:
df_usr_validation = df_usr_view_pay.loc[list(validation_idx)]
df_usr_validation.reset_index(inplace=True)
df_usr_validation.set_index(keys=['shop_id', 'time_stamp'], inplace=True)
df_usr_validation

view_cnt   pay_cnt   weekday     month       aqi  \
shop_id time_stamp                                                     
21      2016-01-12 -0.369900 -0.820407 -1.000197 -1.829243 -0.289440   
        2016-01-13 -0.369900 -0.560338 -0.500967 -1.829243  2.029545   
        2016-01-14 -0.369900 -0.499146 -0.001738 -1.829243  3.368341   
        2016-01-15 -0.369900  0.043939  0.497492 -1.829243  2.866293   
        2016-01-16 -0.369900 -0.323217  0.996722 -1.829243  2.412058   
        2016-01-17 -0.369900 -0.392059  1.495951 -1.829243  0.834192   
        2016-01-18 -0.369900 -0.613882 -1.499427 -1.829243  2.627222   
        2016-01-19 -0.369900 -0.491497 -1.000197 -1.829243  1.073262   
        2016-01-20 -0.369900 -0.537391 -0.500967 -1.829243 -0.791489   
        2016-01-21 -0.369900 -0.353813 -0.001738 -1.829243 -1.078374   
        2016-01-22 -0.369900 -0.116691  0.497492 -1.829243 -0.385069   
        2016-01-23 -0.369900 -0.277322  0.996722 -1.829243 -0.169905   
        2016-01-24 -0.369900 -0.797460  1.495951 -1.829243 -0.050370   
        2016-01-25 -0.369900 -0.828056 -1.499427 -1.829243 -0.122091   
        2016-01-26 -0.369900 -0.828056 -1.000197 -1.829243  0.116980   
        2016-01-27 -0.369900 -0.812758 -0.500967 -1.829243 -0.265533   
        2016-01-28 -0.369900 -0.606233 -0.001738 -1.829243 -0.791489   
        2016-01-29 -0.369900  0.219868  0.497492 -1.829243 -0.791489   
        2016-01-30 -0.369900 -0.040201  0.996722 -1.829243 -0.313348   
        2016-01-31 -0.369900 -0.284971  1.495951 -1.829243  0.164794   
        2016-02-01 -0.016737 -0.384409 -1.499427 -1.510754 -0.122091   
        2016-02-02  0.218704 -0.262024 -1.000197 -1.510754  1.025448   
        2016-02-03  0.124528 -0.315568 -0.500967 -1.510754  0.140887   
        2016-02-04  0.195160 -0.307919 -0.001738 -1.510754 -0.480697   
        2016-02-05 -0.158002 -0.147288  0.497492 -1.510754  2.364244   
        2016-02-06 -0.205091 -0.231428  0.996722 -1.510754  0.379957   
        2016-02-07 -0.252179 -0.529742  1.495951 -1.510754  0.260422   
        2016-02-08 -0.275723 -0.575637 -1.499427 -1.510754  1.168891   
        2016-02-09 -0.275723 -0.460900 -1.000197 -1.510754  0.738563   
        2016-02-10 -0.087370 -0.369111 -0.500967 -1.510754 -0.289440   
...                      ...       ...       ...       ...       ...   
1997    2016-10-02 -0.322812 -0.315568  1.495951  1.037161 -0.863210   
        2016-10-03 -0.299268 -0.384409 -1.499427  1.037161 -0.624139   
        2016-10-04 -0.346356 -0.361462 -1.000197  1.037161 -0.911024   
        2016-10-05 -0.346356 -0.369111 -0.500967  1.037161 -0.385069   
        2016-10-06 -0.299268 -0.193182 -0.001738  1.037161 -0.600232   
        2016-10-07 -0.205091 -0.262024  0.497492  1.037161 -0.648047   
        2016-10-08 -0.134458  0.854743  0.996722  1.037161 -0.815396   
        2016-10-09 -0.299268  1.351933  1.495951  1.037161 -0.743675   
        2016-10-10 -0.228635  1.459021 -1.499427  1.037161 -0.887117   
        2016-10-11 -0.110914  1.566108 -1.000197  1.037161 -0.911024   
        2016-10-12 -0.087370  1.359582 -0.500967  1.037161 -0.576325   
        2016-10-13 -0.228635  1.351933 -0.001738  1.037161 -0.624139   
        2016-10-14 -0.087370  1.336635  0.497492  1.037161 -0.576325   
        2016-10-15 -0.205091 -0.086095  0.996722  1.037161 -0.743675   
        2016-10-16 -0.205091  0.097483  1.495951  1.037161 -0.002556   
        2016-10-17 -0.252179  1.780282 -1.499427  1.037161 -0.456790   
        2016-10-18 -0.275723  1.596704 -1.000197  1.037161 -0.863210   
        2016-10-19 -0.346356  1.512564 -0.500967  1.037161 -0.456790   
        2016-10-20 -0.252179  1.252495 -0.001738  1.037161 -0.815396   
        2016-10-21 -0.275723  0.862392  0.497492  1.037161 -1.126188   
        2016-10-22 -0.205091 -0.162586  0.996722  1.037161 -1.221816   
        2016-10-23 -0.322812  0.074536  1.495951  1.037161 -0.695861   
        2016-10-24 -0.299268  1.382530 -1.499427  1.037161 -

In [13]:
len(df_usr_validation.index.levels[0])

192

# START LSTM~

### Define Data Generator

In [14]:
predict_days = 14
num_pred = 1  ## just predict pay_cnt

In [15]:
class Config(object):
    def __init__(self):
        self.num_steps = 60
        self.batch_size = 200
        self.lstm_size = 64
        self.lstm_layers = 2
        self.feature_size = 83
        self.keep_prob = 0.75
        self.target_size = 1  # pay count 
    def show(self):
        print('num_steps    : {}'.format(self.num_steps))
        print('batch_size   : {}'.format(self.batch_size))
        print('lstm_size    : {}'.format(self.lstm_size))
        print('lstm_layers  : {}'.format(self.lstm_layers))
        print('feature_size : {}'.format(self.feature_size))
        print('keep_prob    : {}'.format(self.keep_prob))
        print('target_size  : {}'.format(self.target_size))
config = Config()

In [16]:
# ss = df_usr_view_pay.loc[1]
# ss.reset_index(drop=True, inplace=True)
# ss

# num_s = 30
# sample1 = ss.loc[0:num_s-1].values
# print(sample1.shape)
# sss = [df_shop_info.loc[1].values]
# sss

# ssss = np.repeat(sss, 30, axis=0)
# ssss
# # np.equal(ssss[0, :], ssss[1, :])

# sample1.shape

# aaa = np.concatenate([ssss, sample1], axis=1)
# print(aaa.shape)
# aaa

In [17]:
def data_generator(config, df_usr, df_shop, random=True):
    batch_size = config.batch_size
    num_steps = config.num_steps
    target_size = config.target_size
    ID_list = list(df_usr.index.levels[0])
    m = len(ID_list)
    assert batch_size <= m, 'batch_size({}) > sample number({}), lower batch_size!'.format(batch_size, m)
    for batch_id in range(int(m/batch_size)): 
        if random:
            np.random.shuffle(ID_list)
            batch_ID_list = ID_list[0:batch_size]
        else:
            batch_ID_list = ID_list[batch_id*batch_size:(batch_id+1)*batch_size]
        _features, _labels = [], []
        for ID in batch_ID_list:
            each_shop_info = [df_shop.loc[ID].values]
            each_shop_usr = df_usr.loc[ID]
            each_shop_usr = each_shop_usr.reset_index(drop=True)
            _m = each_shop_usr.shape[0] - 1
            assert num_steps < _m, 'lower num_steps!'
            _s_idx = np.random.randint(0, _m-num_steps, 1)[0]
            _e_idx = _s_idx + num_steps
#             print('_m: {}, _s_idx: {}, _e_idx: {}'.format(_m, _s_idx, _e_idx))
            f1 = each_shop_usr.loc[_s_idx:_e_idx-1].values
            f1 = np.concatenate([np.repeat(each_shop_info, num_steps, axis=0), f1], axis=1)
            _features.append(f1)
            _labels.append(each_shop_usr.loc[_s_idx+1:_e_idx, 'pay_cnt'].values.reshape(-1, 1))
        
        features = np.stack(_features)
        labels = np.stack(_labels)
        yield features, labels, batch_ID_list


In [18]:
f, l, id_list = data_generator(config, df_usr_view_pay, df_shop_info).__next__()
print(f.shape)
print(l.shape)
print(id_list)

(200, 60, 83)
(200, 60, 1)
[1640, 1737, 642, 1644, 546, 1301, 6, 760, 430, 1800, 1886, 487, 854, 166, 994, 560, 482, 731, 1554, 1542, 1277, 356, 622, 839, 218, 106, 105, 286, 1098, 180, 1493, 1989, 156, 1502, 684, 759, 1307, 910, 1659, 1370, 1291, 888, 767, 682, 777, 1605, 1190, 1814, 1181, 1412, 713, 1374, 911, 1450, 1247, 549, 1932, 447, 265, 927, 1303, 1365, 1631, 1696, 511, 1038, 808, 1555, 262, 645, 165, 1530, 1124, 1589, 439, 922, 152, 1924, 26, 1878, 649, 1470, 811, 593, 122, 1092, 733, 1616, 1265, 1528, 1975, 1421, 536, 1658, 393, 1692, 1016, 1100, 629, 1756, 1089, 146, 505, 1774, 158, 1766, 354, 1741, 1283, 1389, 308, 737, 1952, 1521, 1312, 893, 1556, 1981, 56, 273, 1210, 424, 1258, 992, 329, 1988, 1069, 268, 1410, 1898, 1820, 866, 849, 411, 1396, 1501, 461, 1995, 628, 795, 1407, 1222, 1862, 975, 391, 646, 715, 1263, 1684, 1466, 479, 313, 706, 615, 1173, 485, 627, 1390, 1443, 1019, 605, 633, 1012, 973, 1729, 1456, 1818, 1933, 1567, 781, 147, 1549, 735, 398, 667, 195, 1367, 15,

1. features shape: [batch_size, num_steps, feature_size]
2. labels shape: [batch_size, num_steps]

### Define LSTM

In [19]:
import tensorflow as tf

In [20]:
tf.__version__

'1.0.1'

In [21]:
config.show()

num_steps    : 60
batch_size   : 200
lstm_size    : 64
lstm_layers  : 2
feature_size : 83
keep_prob    : 0.75
target_size  : 1


In [22]:
class LstmModel(object):
    def __init__(self, config, is_training):
        tf.reset_default_graph()
        self._bath_size = batch_size = config.batch_size
        self._feature_size = feature_size = config.feature_size
        self._num_steps = num_steps = config.num_steps
        self._target_size = target_size = config.target_size
        self._lstm_size = lstm_size = config.lstm_size
        self._keep_prob = keep_prob = config.keep_prob
        self._lstm_layers = lstm_layers = config.lstm_layers
        
        self._x = x = tf.placeholder(tf.float32, shape=[batch_size, num_steps, feature_size])
        self._y = y = tf.placeholder(tf.float32, shape=[batch_size, num_steps, target_size])
        
        lstm_cell = tf.contrib.rnn.LSTMCell(lstm_size, forget_bias=0.1)
        if is_training and keep_prob < 1:
            lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
        lstm_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * lstm_layers)
        
        self._initial_state = lstm_cell.zero_state(batch_size, tf.float32)
        
        if is_training and keep_prob < 1:
            x = tf.nn.dropout(x, keep_prob)
        
        outputs = []
        state = self._initial_state
        with tf.variable_scope('lstm'):
            for step in range(num_steps):
                if step > 0:
                    tf.get_variable_scope().reuse_variables()
                cell_output, state = lstm_cell(x[:, step, :], state)
                outputs.append(cell_output)
        self._final_state = state
        outputs = tf.reshape(tf.concat(outputs, axis=1), [-1, lstm_size])
        
        reg_w = tf.get_variable('reg_w', [lstm_size, target_size])
        reg_b = tf.get_variable('reg_b', [target_size])
        self._reg_out = reg_out = tf.matmul(outputs, reg_w) + reg_b
        
#         loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([reg_out], [tf.reshape(y, [-1])], 
#                                                                   [tf.ones([batch_size * num_steps])])
#         self._loss = loss = tf.reduce_sum(loss) / batch_size
        self._loss = loss = tf.reduce_mean(tf.pow(tf.subtract(tf.reshape(reg_out, [-1]),
                                                 tf.reshape(y, [-1])), 2), name='loss')
        if not is_training:
            return

        self._optimizer = tf.train.AdamOptimizer().minimize(loss)

    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y

    @property
    def batch_size(self):
        return self._bath_size

    @property
    def feature_size(self):
        return self._feature_size
    
    @property
    def num_steps(self):
        return self._num_steps
    
    @property
    def optimizer(self):
        return self._optimizer
    
    @property
    def initial_state(self):
        return self._initial_state
    
    @property
    def final_state(self):
        return self._final_state
    
    @property
    def loss(self):
        return self._loss
    
    @property
    def reg_out(self):
        return self._reg_out
        
        

In [23]:
epochs = 50
keep_probablity = 0.75
config.batch_size = 100

In [24]:
config.show()
m = LstmModel(config, is_training=True)

num_steps    : 60
batch_size   : 100
lstm_size    : 64
lstm_layers  : 2
feature_size : 83
keep_prob    : 0.75
target_size  : 1


In [26]:
save_model_path = 'model/customer_prediction'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        new_state = sess.run(m.initial_state)
        for batch_id, (features, labels, _) in enumerate(data_generator(config, df_usr_view_pay, df_shop_info)):
            train_feed_dict = {m.x: features,
                               m.y: labels,
                               m.initial_state: new_state}
            loss, new_state, _ = sess.run([m.loss, m.final_state, m.optimizer], feed_dict=train_feed_dict)
            print('epoch: {:>2}, batch: {:>2}, train_loss: {:>8.4f}'.format(epoch+1, batch_id+1, loss))
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)


epoch:  1, batch:  1, train_loss:   1.0686
epoch:  1, batch:  2, train_loss:   0.5317
epoch:  1, batch:  3, train_loss:   1.1113
epoch:  1, batch:  4, train_loss:   0.6113
epoch:  1, batch:  5, train_loss:   5.6209
epoch:  1, batch:  6, train_loss:   0.4967
epoch:  1, batch:  7, train_loss:   1.1322
epoch:  1, batch:  8, train_loss:   0.4692
epoch:  1, batch:  9, train_loss:   3.9851
epoch:  1, batch: 10, train_loss:   0.3575
epoch:  1, batch: 11, train_loss:   0.7512
epoch:  1, batch: 12, train_loss:   2.0048
epoch:  1, batch: 13, train_loss:   3.5118
epoch:  1, batch: 14, train_loss:   0.5683
epoch:  1, batch: 15, train_loss:   0.3870
epoch:  1, batch: 16, train_loss:   0.6058
epoch:  1, batch: 17, train_loss:   0.3133
epoch:  1, batch: 18, train_loss:   0.2920
epoch:  1, batch: 19, train_loss:   0.2720
epoch:  1, batch: 20, train_loss:   0.3082
epoch:  2, batch:  1, train_loss:   0.5518
epoch:  2, batch:  2, train_loss:   0.2868
epoch:  2, batch:  3, train_loss:   0.2498
epoch:  2, 

# CHECKPOINT

In [27]:
each = df_usr_view_pay.loc[1]
each = each.reset_index(drop=True)
each
each.iloc[-1]

view_cnt   -0.040282
pay_cnt     0.548779
weekday    -1.499427
month       1.037161
aqi        -0.743675
weather    -0.377715
wind        0.245539
tmp        -0.729527
Name: 387, dtype: float64

In [33]:
def test_data_generator(config, df_usr, df_shop, final=False):
    batch_size = config.batch_size
    ID_list = list(df_usr.index.levels[0])
    m = len(ID_list)
    assert batch_size <= m, 'batch_size({}) > sample number({}), lower batch_size!'.format(batch_size, m)
    for batch_id in range(int(m/batch_size)): 
        batch_ID_list = ID_list[batch_id*batch_size:(batch_id+1)*batch_size]
        _features, _labels = [], []
        for ID in batch_ID_list:
            each_shop_info = [df_shop.loc[ID].values]
            each_shop_usr = df_usr.loc[ID]
            if final:
                f1 = each_shop_usr.iloc[-1].values.reshape(1, -1)
                f1 = np.concatenate([np.repeat(each_shop_info, num_steps, axis=0), f1], axis=1)
                _features.append(f1)
            else:
                each_shop_usr = each_shop_usr.reset_index(drop=True)
                _m = each_shop_usr.shape[0] - predict_days
                _idx = np.random.randint(0, _m, 1)[0]
                f1 = each_shop_usr.loc[_idx].values.reshape(1, -1)
                f1 = np.concatenate([each_shop_info, f1], axis=1)
                _features.append(f1)
                _labels.append(each_shop_usr.loc[_idx+1:_idx+predict_days, 'pay_cnt'].values.reshape(-1, 1))
        last_feature = np.stack(_features)
        if final:
            yield last_feature, batch_ID_list
        else:
            gt = np.stack(_labels)
            yield last_feature, gt, batch_ID_list

In [57]:
l, g, b = test_data_generator(test_config, df_usr_view_pay, df_shop_info).__next__()
print(l.shape)
print(g.shape)
print(len(b))
print(g[:, 2:2+1, :].shape)
print(g[:, 2:2+1, :])

(100, 1, 83)
(100, 14, 1)
100
(100, 1, 1)
[[[ 0.83179543]]

 [[-0.20083134]]

 [[-0.0478496 ]]

 [[ 0.18162302]]

 [[ 1.16070617]]

 [[-0.36146217]]

 [[ 0.09748306]]

 [[-0.05549869]]

 [[ 0.41109563]]

 [[-0.78216197]]

 [[-0.49914574]]

 [[-0.82805649]]

 [[-0.33851491]]

 [[ 0.20457028]]

 [[ 0.60232281]]

 [[ 0.41109563]]

 [[-0.26967313]]

 [[-0.05549869]]

 [[ 0.19692119]]

 [[-0.26967313]]

 [[-0.33851491]]

 [[ 0.28106115]]

 [[ 0.03629036]]

 [[-0.46090031]]

 [[-0.56798753]]

 [[-0.07844595]]

 [[-0.43795304]]

 [[ 0.70176094]]

 [[-0.24672586]]

 [[-0.42265487]]

 [[ 2.32336742]]

 [[ 2.19333294]]

 [[-0.39205852]]

 [[ 0.20457028]]

 [[ 3.49367776]]

 [[-0.36146217]]

 [[ 0.42639381]]

 [[-0.46090031]]

 [[-0.82805649]]

 [[-0.16258591]]

 [[ 2.9964871 ]]

 [[-0.32321674]]

 [[-0.49914574]]

 [[-0.13198956]]

 [[-0.45325122]]

 [[-0.20848043]]

 [[-0.2237786 ]]

 [[ 2.91999622]]

 [[ 1.9179658 ]]

 [[-0.39205852]]

 [[-0.61388205]]

 [[-0.43795304]]

 [[ 0.6099719 ]]

 [[ 

In [34]:
test_config = Config()
test_config.num_steps = 1
test_config.batch_size = 100
test_config.keep_prob = 1
test_config.show()

num_steps    : 1
batch_size   : 100
lstm_size    : 64
lstm_layers  : 2
feature_size : 83
keep_prob    : 1
target_size  : 1


In [35]:
m_test = LstmModel(test_config, is_training=False)

In [50]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf

save_model_path = 'model/customer_prediction'

with tf.Session() as sess:
    
    saver = tf.train.Saver()
    saver.restore(sess, save_model_path)

    # Get accuracy in batches for memory limitations
    test_batch_loss_total = 0
    test_batch_count = 0
    result_list = []
    reg_out_list = []
    new_state = sess.run(m_test.initial_state)
    for features, labels, batch_ID_list in test_data_generator(test_config, df_usr_view_pay, df_shop_info):
        test_batch_count += 1
        test_reg_out = []
        for i in range(predict_days):
            if i == 0:
                c = features
            test_feed_dict={m_test.x: c, m_test.y: labels[:, i:i+1, :], m_test.initial_state: new_state}
            test_batch_loss_total += sess.run(m_test.loss, feed_dict=test_feed_dict)
            _test_reg_out = sess.run(m_test.reg_out, feed_dict=test_feed_dict)
            c = _test_reg_out.reshape(-1, 1)
            test_reg_out.append(np.array(_test_reg_out))
        test_reg_out = np.stack(test_reg_out, axis=1)
        tmp_list = [pd.DataFrame(np.array([test_reg_out[i,:], labels[i,:,:]]).T, columns=['pred', 'label']) for i in range(test_config.batch_size)]
        result_list.append(pd.concat(tmp_list, keys=batch_ID_list, names=['shop_id', 'predict_day']))
        reg_out_list.append(pd.DataFrame(test_reg_out))
        
    df_result = pd.concat(result_list)
    
    final_reg_out_list = []
    for features, batch_ID_list in test_data_generator(test_config, df_usr_view_pay, df_shop_info, final=True):
        test_feed_dict={m_test.x: features, m_test.initial_state: new_state}
        final_reg_out = sess.run(m_test.reg_out, feed_dict=test_feed_dict)
        final_reg_out_list.append(pd.DataFrame(final_reg_out))
   
    df_pred = pd.concat(final_reg_out_list, ignore_index=True)
    

    print('Testing Cost: {}\n'.format(test_batch_loss_total/(test_batch_count * predict_days)))


ValueError: Cannot feed value of shape (100, 1) for Tensor 'Placeholder:0', which has shape '(100, 1, 83)'

In [49]:
df_result

NameError: name 'df_result' is not defined

In [ ]:
def negtive_check(df, op=True):
    a = unnormed_df_pred < 0
    if a.sum().sum() > 0:
        print('Find negative!')
        if op:
            df[a] = 0
    else:
        print('no negative number~')

In [ ]:
df_result

In [ ]:
scaled_features

In [ ]:
mean, std = scaled_features['pay_cnt']
unnormed_df_result = df_result *std + mean
unnormed_df_result

In [ ]:
_pred = unnormed_df_result.pred.values
_label = unnormed_df_result.label.values
loss = np.mean(np.abs((_pred-_label)/(_pred+_label)))
loss

### OUTPUT PREDICT

In [ ]:
mean, std = scaled_features['pay_cnt']
unnormed_df_pred = df_pred *std + mean
unnormed_df_pred

In [ ]:
negtive_check(unnormed_df_pred)
unnormed_df_pred


In [ ]:

int_df_pred = unnormed_df_pred.apply(np.round).astype(np.int16)
int_df_pred

int_df_pred.insert(value=range(1, 2001), loc=0, column='id')
int_df_pred

int_df_pred.to_csv('pred/pred_2.csv', header=False, index=False)
int_df_pred

In [ ]:
last_half_month = df_usr_view_pay.loc[(slice(None), slice('2016-10-17', '2016-10-31')), ('pay_cnt')]
mean, std = scaled_features['pay_cnt']
last_half_month = last_half_month * std + mean
last_half_month